## What is Scrapy?

**Scrapy** is a popular Python framework for web scraping and crawling.
It lets you write small programs (called **spiders**) that automatically browse websites and collect data from many pages very quickly.

**Why use Scrapy?**

* It’s **fast** and **efficient** for big projects.
* Handles things like requests, data extraction, and exporting data out of the box.
* Easy to add advanced features (like pipelines and middlewares) as your project grows.

**Real-life Example:**
Imagine you want to collect the names and prices of all books from an online bookstore.
With Scrapy, you can:

* Tell it which website to visit.
* Show it where to find the name and price on each page.
* Scrapy will visit every book page, collect the data, and save it to a file for you—**automatically**.

---

## **Full Flow with Scrapy**

Here’s how it all connects:

1. **Scrapy Spider**: You write a spider that tells Scrapy what website to visit and what data to collect.
2. **Pipelines**: Scrapy passes the collected data through pipelines to clean or save it.
3. **Middlewares**: Scrapy uses middlewares to add extra rules (like delays, proxies, user-agent changes) while browsing.
4. **Data Export**: Scrapy saves all your final, clean data into files (CSV, JSON, etc.), ready for you to use.

---

## Visual Diagram (Text Form)

```
[Website]
    ↓
[Scrapy Spider] → [Middlewares] → [Pipelines] → [Export File (CSV/JSON)]
```

---

**In Short:**

* **Scrapy** is the engine.
* **Spider** does the crawling.
* **Middlewares** are helpers on the way.
* **Pipelines** clean and save the results.
* **Data export** gives you the final output.


## Goal:
I want to collect the title and price of all books from an online bookstore, and save them to a CSV file.
